In [27]:
from langchain_gigachat import GigaChat
import os
import torch
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import pandas as pd
import re
from typing import List
from sentence_transformers import CrossEncoder
from langchain_core.output_parsers import StrOutputParser

# Проверяем доступность GPU
print(f"CUDA доступен: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    device = "cuda"
else:
    print("GPU не доступен, используем CPU")
    device = "cpu"

# Настройка GROQ API ключа
GROQ_API_KEY = "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"
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY не найден. Убедитесь, что он указан в .env файле")

# Путь к директории с текстовыми документами и векторному хранилищу
DOCUMENTS_DIR = "/content/data"
VECTOR_DB_PATH = "/content/drive/MyDrive/chroma_db_1"
types = set()
def load_documents(directory):
    """Загружает все текстовые документы из указанной директории и добавляет имя файла в метаданные"""
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            try:
                loader = TextLoader(file_path)
                docs = loader.load()

                # Добавляем имя файла в метаданные
                for doc in docs:
                    metadatas = doc.page_content.split('\n')[0:3]
                    metadatas = [x.split(':',1)[1].lstrip() for x in metadatas]
                    doc.metadata["Источник"] = metadatas[0]
                    doc.metadata["Тип"] = metadatas[1]
                    doc.metadata["Модель"] = metadatas[2]
                    types.add(metadatas[1])
                    print(doc.metadata)
                documents.extend(docs)
            except Exception as e:
                print(f"Ошибка при загрузке {filename}: {str(e)}")
    return documents

def create_vectorstore(documents):
    """Создает и сохраняет векторное хранилище из документов, используя GPU"""
    # Разбиваем документы на чанки
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=30,
        chunk_overlap=0,
        length_function=len,
        separators=[';',"\n"]
    )
    split_docs = text_splitter.split_documents(documents)

    # Создаем эмбеддинги с использованием GPU
    model_kwargs = {'device': device}
    encode_kwargs = {'device': device, 'batch_size': 128}  # Увеличиваем batch_size для GPU

    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-large',
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    print(f"Создание векторного хранилища на {device}...")
    # Создаем векторное хранилище
    vectorstore = Chroma.from_documents(
        documents=split_docs,
        embedding=embeddings,
        persist_directory=VECTOR_DB_PATH
    )

    # Сохраняем хранилище на диск
    vectorstore.persist()
    print(f"Векторное хранилище создано и сохранено в {VECTOR_DB_PATH}")

    return vectorstore

def load_vectorstore():
    """Загружает существующее векторное хранилище"""
    # Настраиваем модель эмбеддингов для использования GPU
    model_kwargs = {'device': device}
    encode_kwargs = {'device': device, 'batch_size': 32}

    embeddings = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-large',
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )

    # Загружаем существующее хранилище
    vectorstore = Chroma(
        persist_directory=VECTOR_DB_PATH,
        embedding_function=embeddings
    )

    print(f"Векторное хранилище загружено из {VECTOR_DB_PATH}")
    return vectorstore

# Остальной код остается без изменений
def setup_rag_system():
    """Настраивает RAG систему, загружая или создавая векторное хранилище"""
    # Проверяем, существует ли уже векторное хранилище
    if os.path.exists(VECTOR_DB_PATH) and os.listdir(VECTOR_DB_PATH):
        # Если хранилище существует, загружаем его
        vectorstore = load_vectorstore()
        load_documents(DOCUMENTS_DIR)
    else:
        # Если хранилища нет, создаем новое
        print("Векторное хранилище не найдено. Создаем новое...")
        documents = load_documents(DOCUMENTS_DIR)
        if not documents:
            raise ValueError("Не удалось загрузить ни одного документа!")
        vectorstore = create_vectorstore(documents)

    # Настраиваем GROQ LLM
    llm = ChatGroq(
        groq_api_key=GROQ_API_KEY,
        model_name="llama3-70b-8192",
        temperature=0.3,
        max_tokens=1400,
    )

    return vectorstore, llm

# Остальной код без изменений...


try:
    # Настраиваем RAG систему (загружаем существующее хранилище или создаем новое)
    print("Настройка RAG системы...")
    vectorstore, llm = setup_rag_system()


except Exception as e:
    print(f"Произошла ошибка: {str(e)}")

# Если нужно обновить хранилище (например, при добавлении новых документов):
# update_vectorstore()



print(types)






CUDA доступен: False
GPU не доступен, используем CPU
Настройка RAG системы...
Векторное хранилище загружено из /content/drive/MyDrive/chroma_db_1
{'source': '/content/data/mini-pec-dexp-vn-3800-cernyj.txt', 'Источник': 'https://www.dns-shop.ru/product/78b655e9fa0ded20/mini-pec-dexp-vn-3800-cernyj/characteristics/', 'Тип': 'мини-печь', 'Модель': 'DEXP VN-3800'}
{'source': '/content/data/masorubka-elektriceskaa-redmond-rmg-1203-8-belyj.txt', 'Источник': 'https://www.dns-shop.ru/product/a907340142b03120/masorubka-elektriceskaa-redmond-rmg-1203-8-belyj/characteristics/', 'Тип': 'мясорубка электрическая', 'Модель': 'Redmond RMG-1203-8'}
{'source': '/content/data/multivarka-polaris-pmc-0573ad-serebristyj.txt', 'Источник': 'https://www.dns-shop.ru/product/72ca334cd5371b80/multivarka-polaris-pmc-0573ad-serebristyj/characteristics/', 'Тип': 'мультиварка', 'Модель': 'Polaris PMC 0573AD'}
{'source': '/content/data/holodilnik-kompaktnyj--birusa-70-belyj.txt', 'Источник': 'https://www.dns-shop.ru/p

In [10]:
!zip -r my_archive.zip chroma_db

  adding: chroma_db/ (stored 0%)
  adding: chroma_db/d39736d6-bf89-478b-b83d-92aa1a2ee1cb/ (stored 0%)
  adding: chroma_db/d39736d6-bf89-478b-b83d-92aa1a2ee1cb/index_metadata.pickle (deflated 43%)
  adding: chroma_db/d39736d6-bf89-478b-b83d-92aa1a2ee1cb/header.bin (deflated 56%)
  adding: chroma_db/d39736d6-bf89-478b-b83d-92aa1a2ee1cb/data_level0.bin (deflated 10%)
  adding: chroma_db/d39736d6-bf89-478b-b83d-92aa1a2ee1cb/link_lists.bin (deflated 83%)
  adding: chroma_db/d39736d6-bf89-478b-b83d-92aa1a2ee1cb/length.bin (deflated 90%)
  adding: chroma_db/chroma.sqlite3 (deflated 93%)


In [2]:
!unzip "my_archive (5).zip" -d chroma_db

Archive:  my_archive (5).zip
   creating: chroma_db/chroma_db1/
  inflating: chroma_db/chroma_db1/chroma.sqlite3  
   creating: chroma_db/chroma_db1/72068996-67be-4503-8250-d70535372315/
  inflating: chroma_db/chroma_db1/72068996-67be-4503-8250-d70535372315/index_metadata.pickle  
  inflating: chroma_db/chroma_db1/72068996-67be-4503-8250-d70535372315/header.bin  
  inflating: chroma_db/chroma_db1/72068996-67be-4503-8250-d70535372315/data_level0.bin  
  inflating: chroma_db/chroma_db1/72068996-67be-4503-8250-d70535372315/link_lists.bin  
  inflating: chroma_db/chroma_db1/72068996-67be-4503-8250-d70535372315/length.bin  


In [30]:
from langchain_gigachat import GigaChat
import os
import torch
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import pandas as pd
import re
from typing import List
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata
from IPython.display import display, Markdown
from collections import Counter
import time
creds = userdata.get('GigaChatKey')

# llm = GigaChat(
#         temperature=0.2,
#         max_tokens=700,
#         verify_ssl_certs=False,
#         credentials=creds,
#         scope='GIGACHAT_API_PERS',
#         model="GigaChat-Max"
#     )

llm = ChatGroq(
        groq_api_key=GROQ_API_KEY,
        model_name="llama3-70b-8192",
        temperature=0.5,
        max_tokens=1400,
    )


get_type_prompt = PromptTemplate(
    template = """
    Запрос пользователя: {msg}

    Определи, какие типы товаров из доступных подойдут к этому запросу пользователя.
     В ответе пиши ТОЛЬКО типы товаров в точности как я дал тебе, другие типы писать не разрешается.
     Пиши в строчку через запятую. Если нет подходящих товаров, то напиши "такой техники нет".
     ПИШИ ТОЛЬКО ПОДХОДЯЩИЕ ПОД ЗАПРОС ТИПЫ ТОВАРОВ. НЕ ПИШИ БОЛЕЕ 3 ТИПОВ.
     Доступные типы товаров: {types}
     Подходящие типы товаров:
    """,
    input_variables = ['msg','types']
)

get_type_chain = get_type_prompt|llm|StrOutputParser()
#проверим как оно работает
##print(get_type_chain.invoke({'msg':"нужен слон","types":types}))
##print(get_type_chain.invoke({'msg':"нужна штука которая делает холод короче и туда еще еду можно пихать и чтоб большая и крутая","types":types}))
#добавим получение характеристик из 'шаблона'
msg = "хочу купить слона"
searched_types = get_type_chain.invoke({'msg':msg,"types":', '.join(types)}).split(', ')
#такие типы подходящих товаров выходят
print(searched_types)
#сюда 'шаблоны' (примеры документа с товаром) пишем
product_examples = []
#накидываем шаблоны
for x in searched_types:

  if len(vectorstore.get(where={"Тип":x},limit=1)['ids']):
    filepath = vectorstore.get(where={"Тип":x},limit=1)['metadatas'][0]['source']
  else:
    continue
  with open(filepath,'r') as f:
    product_examples.append(f.read())

product_examples=["Пример "+str(x)+":"+y+'\n\n' for x,y in enumerate(product_examples)]
print(product_examples)
get_chars_prompt = PromptTemplate(
    template="""
    Пожелания пользователя:
    {msg}

    По пожеланиям определи, какие характеристики важны для пользователя.
    Напиши эти характеристики в формате название1: значение1\n название2: значение2\n и т.д.
    Пиши ТОЛЬКО ПАРЫ. Старайся угадать примерные ЧИСЛЕННЫЕ (там где они могут быть численными) значения.
     названия характеристик бери из данных тебе примеров, строго соблюдай их.
     Значения ты должен приблизительно оценить исходя из пожеланий пользователя.
      Пиши конкретные значения, без уточнений, сносок. Если характеристика не относится к пожеланиям пользователя то ты ее не пишешь.


    Примеры характеристик (значения могут не соответствовать пожеланиям):
    {examples}

    """,
    input_variables = ['msg','examples']
)

get_chars_chain = get_chars_prompt|llm|StrOutputParser()
#работает?
chars = get_chars_chain.invoke({'msg':msg,'examples':product_examples}).split('\n')
print(chars)

#добавим сообщение пользователя для поиска по описанию
chars.append(msg)
#ищем по этим характеристикам
filter = {"Тип": {"$in": searched_types}}
suiting_models = []
for char in chars:
  res = vectorstore.similarity_search(char,filter=filter,k=10)
  models = {x.metadata['Модель'] for x in res}
  suiting_models+=models
  print(suiting_models)

chosen_models=Counter(suiting_models).most_common(4)
print(Counter(suiting_models))
get_recomendations_template=PromptTemplate(
    template="""
    Пользователь хочет найти товар подходящий под его описание. СТРОГО следуй такому плану ответа:
    1. Опиши каждый подходящий запросу пользователя товар, дай его необходимые характеристики и немного дополнительных, опиши его плюсы и минусы (ОТНОСИТЕЛЬНО ЗАПРОСА ПОЛЬЗОВАТЕЛЯ), причины, по которым ты выбрал этот товар как подходящий.
    2. Основываясь на своих объективных рассуждениях (на основе характеристик подходящих товаров), сравни подходящие товары между собой (ОТНОСИТЕЛЬНО ЗАПРОСА ПОЛЬЗОВАТЕЛЯ), дай пользователю рекомендации.
    3. ЕСЛИ ТОВАР НЕ ПОДХОДИТ ПОД ЗАПРОС ПОЛЬЗОВАТЕЛЯ, ТЫ ЕГО НЕ ПИШЕШЬ.
    Запрос пользователя: {msg}

    Характеристики товаров: {products}

    Подходящие товары, их описания с плюсами и минусами, их сравние и рекомендации на русском:
    """,
    input_variables=['msg','products']
)

get_recomendations_chain = get_recomendations_template|llm|StrOutputParser()

products = []
print(chosen_models)
for x in chosen_models:
  filepath = vectorstore.get(where={"Модель":x[0]},limit=1)['metadatas'][0]['source']
  with open(filepath,'r') as f:
    products.append(f'Модель - {x}\n'+f.read())
products = '\n'.join(products)

display(Markdown(get_recomendations_chain.invoke({'msg':msg,'products':products})))

['Такой техники нет.']
[]
['Вот какие характеристики важны для пользователя:', '', 'Вес: 5000 кг', 'Высота: 3 м', 'Цена: 1000000 руб', 'Возраст: 5 лет', 'Пол: мужской', 'Порода: индийский слон']
[]
[]
[]
[]
[]
[]
[]
[]
[]
Counter()
[]


**Подходящие товары**

1. **Фигурка слона из керамики**
	* Характеристики: материал - керамика, размер - 15 см, вес - 0,5 кг, цвет - белый
	* Плюсы: декоративный, безопасный, легко очищается
	* Минусы: не может быть использован как игрушка, может быть хрупким
	* Причина выбора: это декоративный элемент, который может быть использован для украшения интерьера, и он соответствует запросу пользователя о покупке слона
2. **Слон из плюша**
	* Характеристики: материал - плюш, размер - 30 см, вес - 0,2 кг, цвет - серый
	* Плюсы: мягкий, приятный на ощупь, может быть использован как игрушка
	* Минусы: может потерять форму, не подходит для украшения интерьера
	* Причина выбора: это мягкая игрушка, которая может быть использована для игры или как декоративный элемент, и она соответствует запросу пользователя о покупке слона

**Сравнение товаров**

Оба товара соответствуют запросу пользователя о покупке слона, но они имеют разные характеристики и предназначения. Фигурка слона из керамики является декоративным элементом, который может быть использован для украшения интерьера, а слон из плюша является мягкой игрушкой, которая может быть использована для игры или как декоративный элемент.

**Рекомендации**

Если вы ищете декоративный элемент для украшения интерьера, то рекомендуется выбрать фигурку слона из керамики. Если вы ищете мягкую игрушку для игры или как декоративный элемент, то рекомендуется выбрать слона из плюша.

In [130]:
from langchain.tools import Tool
from collections import Counter
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt.chat_agent_executor import AgentState
import langgraph
import os
from langchain.agents import AgentExecutor, create_react_agent
import os
from dotenv import load_dotenv
from typing import List, Optional
from operator import itemgetter
import time

llm = ChatGroq(
        groq_api_key=GROQ_API_KEY,
        model_name="llama3-70b-8192",
        temperature=0.5,
        max_tokens=500,
    )

llm2 = GigaChat(
        temperature=0.2,
        max_tokens=700,
        verify_ssl_certs=False,
        credentials=creds,
        scope='GIGACHAT_API_PERS',
   )

get_recomendations_template=PromptTemplate(
    template="""
    Пользователь хочет найти товар подходящий под его описание. СТРОГО следуй такому плану ответа:
    1. Опиши каждый подходящий запросу пользователя товар, дай его необходимые характеристики и немного дополнительных, опиши его плюсы и минусы (ОТНОСИТЕЛЬНО ЗАПРОСА ПОЛЬЗОВАТЕЛЯ), причины, по которым ты выбрал этот товар как подходящий.
    2. Основываясь на своих объективных рассуждениях (на основе характеристик подходящих товаров), сравни подходящие товары между собой (ОТНОСИТЕЛЬНО ЗАПРОСА ПОЛЬЗОВАТЕЛЯ), дай пользователю рекомендации.
    3. ЕСЛИ ТОВАР НЕ ПОДХОДИТ ПОД ЗАПРОС ПОЛЬЗОВАТЕЛЯ, ТЫ ЕГО НЕ ПИШЕШЬ.
    Запрос пользователя: {msg}

    Характеристики товаров: {products}

    Подходящие товары, их описания с плюсами и минусами, их сравние и рекомендации на русском:
    """,
    input_variables=['msg','products']
)

get_type_prompt = PromptTemplate(
    template = """
    Запрос пользователя: {msg}

    Определи, какие типы товаров из доступных подойдут к этому запросу пользователя.
     В ответе пиши ТОЛЬКО типы товаров в точности как я дал тебе, другие типы писать не разрешается.
     Пиши в строчку через запятую. Если нет подходящих товаров, то напиши "нет".
     ПИШИ ТОЛЬКО ПОДХОДЯЩИЕ ПОД ЗАПРОС ТИПЫ ТОВАРОВ. НЕ ПИШИ БОЛЕЕ 3 ТИПОВ.
     Доступные типы товаров: {types}
     Подходящие типы товаров:
    """,
    input_variables = ['msg','types']
)

get_chars_prompt = PromptTemplate(
    template="""
    Пожелания пользователя:
    {msg}

    По пожеланиям определи, какие характеристики важны для пользователя.
    Напиши эти характеристики в формате название1: значение1\n название2: значение2\n и т.д.
    Пиши ТОЛЬКО ПАРЫ. Старайся угадать примерные ЧИСЛЕННЫЕ (там где они могут быть численными) значения.
     названия характеристик бери из данных тебе примеров, строго соблюдай их.
     Значения ты должен приблизительно оценить исходя из пожеланий пользователя.
      Пиши конкретные значения, без уточнений, сносок. Если характеристика не относится к пожеланиям пользователя то ты ее не пишешь.


    Примеры характеристик (значения могут не соответствовать пожеланиям):
    {examples}

    """,
    input_variables = ['msg','examples']
)

get_type_chain = get_type_prompt|llm|StrOutputParser()
get_chars_chain = get_chars_prompt|llm|StrOutputParser()
get_recomendations_chain = get_recomendations_template|llm|StrOutputParser()

def get_suiting_product_data(query:str)->str:
  searched_types = get_type_chain.invoke({'msg':query,"types":', '.join(types)}).split(', ')
  #проверка на наличие
  if not len(searched_types) or searched_types[0]=='нет':
    return f"Кажется, нет типов товаров, подходящих под данный запрос. Доступные типы товаров : {', '.join(types)}"
  #сюда 'шаблоны' (примеры документа с товаром) пишем
  product_examples = []
  #накидываем шаблоны
  for x in searched_types:
    if len(vectorstore.get(where={"Тип":x},limit=1)['ids']):
      filepath = vectorstore.get(where={"Тип":x},limit=1)['metadatas'][0]['source']
    else:
      continue
    with open(filepath,'r') as f:
      product_examples.append(f.read())
  #форматируем в роботочитаемый текст
  product_examples=["Пример "+str(x)+":"+y+'\n\n' for x,y in enumerate(product_examples)]
  #теперь найдем примерные характеристики товара по шаблону
  chars = get_chars_chain.invoke({'msg':query,'examples':product_examples}).split('\n')
  #добавим сообщение пользователя для поиска по описанию
  chars.append(query)
  #ищем по этим характеристикам
  filter = {"Тип": {"$in": searched_types}}
  suiting_models = []
  for char in chars:
    res = vectorstore.similarity_search(char,filter=filter,k=10)
    models = {x.metadata['Модель'] for x in res}
    suiting_models+=models
  #тут мы короче сделали запрос по каждой характеристике, посчитали кол-во вхождений одной и той же модели
  #(кол-во попадающих под запрос характеристик)
  products =[]
  chosen_models=[x[0] for x in Counter(suiting_models).most_common(3)]
  print(chosen_models)
  for x in chosen_models:
    filepath = vectorstore.get(where={"Модель":x},limit=1)['metadatas'][0]['source']
    with open(filepath,'r') as f:
      products.append(f'Модель - {x}\n'+f.read())
  products = '\n'.join(products)
  return products

def get_product_by_model(model:str)->str:
  filter = {"Модель": model}
  if len(vectorstore.get(where={"Модель":model},limit=1)['metadatas']):
    filepath = vectorstore.get(where={"Модель":model},limit=1)['metadatas'][0]['source']
    with open(filepath,'r') as f:
      return f'Точное совпадение, Модель - {x}\n'+f.read()
  else:
    results = [x.metadata['Модель'] for x in vectorstore.similarity_search(f'Модель: {model}',k=5)]
    print(results)
    return "Не найдено точное совпадение, возможно данные отрывки текста смогут помочь?\n"+', '.join(results)




# Создаем объекты инструментов
tools = [
    Tool(
        name="GetSuitingProducts",
        func=lambda query: get_suiting_product_data(query),
        description="Принимает на вход ЗАПРОС (СТРОКУ). Находит подходящие по запросу товары. Указание конкретных характеристик повысит точность."
    ),
        Tool(
        name="GetProductByModel",
        func=lambda model: get_product_by_model(model),
        description="На вход принимает СТРОКУ с названием модели бытовой техники. Возвращает (СТРОКУ) характеристики техники в случае точного совпадения. Иначе вернет список похожишь значений"
    )
]

# Промпт для агента
prefix = """
Ты — ассистент по подбору бытовой техники. Твоя задача:
1. Найди информацию, связанную с запросом пользователя. Или попроси пользователя уточнить запрос.
2. Выбери подходящую под запрос пользователя технику. Не подходящую не показывай
3. Опиши характеристики подходящей техники. Ее плюсы и минусы, дай рекомендацию пользователю на основе данных.
4(опционально). Уточни информацию, если пользователь попросит.
функция GetSuitingProducts - принимает запрос (query) для поиска данных. Ты можешь уточнять запрос по просьбе пользователя, старайся использовать эту функцию реже.
функция GetProductByModel - принимает модель техники. Возвращает точное описание модели техники при точном совпадении иначе - список похожих моделей.
 Используй GetProductByModel для уточнения характеристик ранее упомянутых тобою моделей техники, если потребуется.
"""

#agent.run('хочу пылесос')
memory = MemorySaver()
agent_executor = langgraph.prebuilt.chat_agent_executor.create_react_agent(llm, tools, checkpointer=memory,debug=False,prompt=prefix)
config = {"configurable": {"thread_id": "abc123"}}

result = agent_executor.invoke({
    "messages": [{"role": "user", "content": "Хочу пылесос"}]
}, config)
print("запрос 1: Хочу пылесос")
display(Markdown(result["messages"][-1].content))
time.sleep(60)
result = agent_executor.invoke({
    "messages": [{"role": "user", "content": "какой из представленных тобою ранее пылесосов по твоему мнению лучший?"}]
}, config)
print("запрос 2: какой из представленных тобою ранее пылесосов по твоему мнению лучший?")
display(Markdown(result["messages"][-1].content))
time.sleep(60)
result = agent_executor.invoke({
    "messages": [{"role": "user", "content": "Среди упомянутых тобою пылесосов найди тот, у которого наибольшая длина провода"}]
}, config)
print("запрос 3: Среди упомянутых тобою пылесосов найди тот, у которого наибольшая длина провода")
display(Markdown(result["messages"][-1].content))

['Centek CT-2519', 'Bosch BGBS2BA1P', 'Delta LUX DE-3501']
запрос 1: Хочу пылесос


Вот характеристики пылесоса Centek CT-2519, который может быть подходящим вариантом для вас. Он имеет фильтр тонкой очистки, который эффективно удаляет аллергены и загрязнения из воздуха. Емкость пылесборника составляет 4 л, а регулятор мощности расположен на корпусе для удобства пользователей. Телескопическая труба всасывания легко подстраивается под рост пользователя, а технология защиты от перегрева не позволяет прибору внезапно выйти из строя. Комплект насадок позволяет легко удалять загрязнения из труднодоступных мест. 

В целом, это хороший вариант для сухой уборки помещений. Если у вас есть вопросы или вам нужно больше информации, пожалуйста, уточните.

запрос 2: какой из представленных тобою ранее пылесосов по твоему мнению лучший?


Сравнивая характеристики пылесосов Centek CT-2519, Bosch BGBS2BA1P и Delta LUX DE-3501, я бы рекомендовал Centek CT-2519. Он имеет фильтр тонкой очистки, который эффективно удаляет аллергены и загрязнения из воздуха, а также имеет телескопическую трубу всасывания, которая легко подстраивается под рост пользователя. Кроме того, он имеет регулятор мощности на корпусе, что удобно для пользователей. Емкость пылесборника составляет 4 л, что достаточно для большинства случаев.

Однако, если вам важна производительность и мощность, то Bosch BGBS2BA1P может быть более подходящим вариантом. Он имеет более высокую мощность всасывания и более эффективную систему фильтрации.

Delta LUX DE-3501 также имеет свои преимущества, такие как 5-ступенчатая система фильтрации и циклонная система фильтрации, но он имеет меньшую емкость пылесборника и меньшую мощность всасывания.

В целом, выбор лучшего пылесоса зависит от ваших конкретных потребностей и предпочтений.

запрос 3: Среди упомянутых тобою пылесосов найди тот, у которого наибольшая длина провода


Сравнивая характеристики пылесосов, я нашел, что у Bosch BGBS2BA1P длина сетевого шнура составляет 7 м, что является наибольшей среди упомянутых пылесосов.

In [110]:
!pip install --upgrade langgraph

In [1]:

!pip install --upgrade langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.20
    Uninstalling langchain-0.3.20:
      Successfully uninstalled langchain-0.3.20


In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 4.8 MB/s eta 0:0

In [3]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.45
    Uninstalling langchain-core-0.3.45:
      Successfully uninstalled langchain-core-0.3.45


In [4]:
!pip install --upgrade huggingface_hub transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 23.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0


In [5]:
!pip install langchain_community

In [6]:
!pip install --upgrade langchain_groq

In [ ]:
def generate_answer(row):

    filter = {"filename": {"$in": ["Война и мир 1.txt", "Война и мир 2.txt"]}}
    res = vectorstore.similarity_search(q,filter=filter,k=1)
    context.append(res[0].page_content)
  vectorstore.similarity_search(question,filter=filter,k=1)[0].page_content)

In [ ]:
for var in globals().copy():
    if var.startswith("__"):
        continue
    del globals()[var]

for var in locals().copy():
    if var.startswith("__"):
        continue
    del locals()[var]


In [7]:
!pip install langchain_gigachat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00


In [52]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.2 MB/s eta 0:00:00
